---
title: "Meta Learning in Neural Networks — A Survey"
date: 2025-11-14       
description: "메타 러닝 관련 논문 요약 및 주요 내용"
categories: [MetaLearning, Survey, Review]
author: "김한울"
---


# Paper Review: Meta Learning in Neural Networks: A Survey

``` 
@article{hospedales2021meta,
  title={Meta-learning in neural networks: A survey},
  author={Hospedales, Timothy and Antoniou, Antreas and Micaelli, Paul and Storkey, Amos},
  journal={IEEE transactions on pattern analysis and machine intelligence},
  volume={44},
  number={9},
  pages={5149--5169},
  year={2021},
  publisher={IEEE}
}
```

## 초록

`Learning-to-learn`이라고도 불리는 메타-러닝 분야는 최근 몇 년간 관심이 급격히 증가해왔습니다. 
고정된 학습 알고리즘을 사용하여 처음부터 과제를 해결하는 기존의 AI 접근 방식과는 대조적으로, 
메타-러닝은 다수의 학습 에피소드 경험을 바탕으로 **학습 알고리즘 자체를 개선하는 것을 목표**로 합니다. 
이 패러다임은 데이터 및 계산 병목 현상, 그리고 일반화 성능 등 딥러닝의 여러 기존 난제들을 해결할 기회를 제공합니다. 
본 서베이는 현대 메타-러닝의 전반적인 동향을 기술합니다. 먼저 메타-러닝의 정의를 논하고, 전이 학습(transfer learning) 및 하이퍼파라미터 최적화(hyperparameter optimization)와 같은 관련 분야와 비교하여 그 위치를 정립합니다. 
다음으로, 오늘날의 메타-러닝 방법 공간을 더 포괄적으로 분석하는 새로운 분류 체계를 제안합니다. 
또한 퓨샷 러닝(few-shot learning) 및 강화 학습(reinforcement learning)과 같은 메타-러닝의 유망한 응용 분야와 성공 사례들을 살펴봅니다. 마지막으로, 아직 해결되지 않은 과제들과 향후 연구를 위한 유망한 영역들에 대해 논의합니다.

## Introduction

메타러닝이란, "머신러닝 모델"이 "공부하는 법"자체를 배우게 하는 새로운 패러다임이라고 할 수 있음

> Meta-learning provides an alternative paradigm where a machine learning model gains experience over multiple learning episodes – often covering a distribution of related tasks – and uses this experience to improve its future learning performance.

1. provides an alternative paradigm?

* 기존 방식?    
  기계에게 '고양이 사진 분류'라는 숙제 하나를 주고, 그 숙제만 잘 풀도록 처음부터 끝까지 가르칩니다. 다른 숙제(예: '개 사진 분류')를 주면 또 처음부터 새로 배워야 합니다.   
* 대안(메타러닝):    
  기계에게 숙제 하나만 주는 게 아니라, **"스스로 공부하는 법"**을 터득하게 만듭니다.   

2. a machine learning model gains experience over multiple learning episodes

* 비유를 해보자면    
  사람에게 수학만 가르치는 게 아니라, 수학, 과학, 국어 등 **여러 과목의 문제(관련된 과제들)**를 풀게 하는 행위를 일컫는 문장입니다.   
  이 과정에서 모델은 단순히 개별 문제를 푸는 법을 배우는 게 아니라, 문제들 사이의 공통점이나 문제 해결의 **'요령(패턴)'**을 깨닫게 됩니다. 이것이 바로 **"경험"**이라고 할 수 있습니다.
* `learning episodes`: 문제 하나하나를 풀어보는 경험 한 번 한 번을 의미합니다.


보통의 경우에, 그리고 역사적으로 보면, 

1. 머신러닝   
+ 기계가 데이터를 잘 이해할 수 있도록 데이터의 핵심적인 특징(feature)을 사람 전문가가 직접 뽑아냅니다.
  + hand-engineered features   
+ 그 후, 사람이 뽑아낸 특징 데이터를 입력으로 받아서, 기계가 정답을 맞히는 패턴을 학습합니다.
  + 이미지 문제를 예로 들었을 때, 기계는 원본 이미지를 보고 있는 것이 아니라, 사람이 가공해서 준 특징 값들만 보게 됩니다.   
+ 즉, 머신러닝에서 **특징 추출 단계**와 **모델 학습 단계**는 분리 되어 있습니다.

2. 딥러닝   
+ 위의 두 단계인 **특징 추출 단계**와 **모델 학습 단계**를 하나의 단계로 통합했습니다.    
  + 특징과 모델의 공동 학습 (Joint feature and model learning)    
+ 예전 처럼 특징을 정성스럽게 뽑아낼 필요가 없습니다.
  + 원본 데이터(이미지 형태 그대로!) 모델에 그대로 입력합니다.   
  + 그러면 모델이 내부의 여러 계층(layer)을 거치면서 자동으로 특징을 찾아내고, 동시에 그 특징을 이용해 분류하는 방법까지 한꺼번에 학습합니다.   
+ 고양이 사진 분류하는 모델을 예로 들어보겠습니다.
  + 초반 계층: 이미지의 가장 기본적인 특징(선, 색상, 명암 대비 등)을 스스로 학습합니다.   
  + 중간 계층: 초반 계층에서 학습한 기본 특징들을 조합하여 더 복잡한 특징(눈, 코, 귀의 형태 등)을 학습합니다.   
  + 후반 계층: 중간 계층의 복잡한 특징들을 다시 조합하여 최종적으로 "이것이 고양이다"라고 판단하는 방법을 학습합니다.
+ 어떤 특징이 중요한지를 기계가 데이터로부터 직접 배우는 것입니다. 사람이 "귀가 중요해, 수염이 중요해"라고 알려줄 필요가 없습니다. 특징을 배우는 과정과 그 특징으로 정답을 맞히는 과정이 '공동으로(jointly)', 즉 '동시에' 최적화됩니다.   

3. 신경망에서의 메타러닝?
+ 특징 추출 단계, 모델 학습 단계, 그리고 `알고리즘`을 하나의 단계로 통합하고자 하는 학습 방식이라고 할 수 있습니다.

메타러닝은 

+ 과제군(a family of tasks)에서 과제에 구애받지 않는(task-agnostic) 지식을 추출하여 해당 과제군 내의 새로운 과제 학습을 개선하는 다중 과제 시나리오에서도 유용성을 입증했고,   
+ 단일 문제를 반복적으로 해결하며 여러 에피소드에 걸쳐 개선해 나가는 단일 과제 시나리오에서도 유용성이 입증되었습니다. 

성공적인 응용 사례: 퓨샷 이미지 인식, 비지도 학습, 데이터 효율 개선 및 자기 주도적 강화 학습(RL), 하이퍼파라미터 최적화, 신경망 구조 탐색(NAS) 등.


Thrun은 '러닝-투-런'을 다음과 같이 측정 가능하게(operationally) 정의하고 있습니다. 

> Thrun [7] operationally defines learning-to-learn as occurring when a learner’s performance at solving tasks drawn from a given task family improves with respect to the number of tasks seen.   
  > *해당 책은 한 권에 40만원이라 읽어보지는 못할 것 같다. S. Thrun and L. Pratt, “Learning To Learn: Introduction And
Overview,” in Learning To Learn, 1998 *

> 학습자(AI)가 관련된 과제 그룹(task family)에서 여러 과제를 풀어볼수록,    
> 즉 더 많은 종류의 과제를 경험할수록 새로운 과제를 푸는 성능이 향상되는 현상.

+ task family (과제 그룹): 서로 다르지만 관련이 있는 문제들의 묶음입니다.   
  + 예시: ('고양이/개 분류', '사자/호랑이 분류', '새/물고기 분류')는 모두 **"동물 분류"**라는 하나의 task family에 속합니다.   
  + AI가 '고양이/개 분류' 문제를 푼 다음, '사자/호랑이 분류' 문제를 풀고, 또 '새/물고기 분류' 문제를 푸는 등... 이렇게 다양한 종류의 과제(tasks)를 더 많이 경험할수록, 나중에 처음 보는 '코끼리/기린 분류' 문제도 더 잘 풀게 된다는 것입니다. 
  + 즉, 경험하는 과제의 '종류'가 늘어날수록 똑똑해집니다.

> conventional machine learning performance improves as more data from a single task is seen

+ 기존 머신러닝은 하나의 과제(a single task)에 대한 데이터가 많아질수록 성능이 향상됩니다.

**공짜 점심은 없다(no free lunch) Theorem** 에 대항하는 알고리즘 ㅋㅋ

> This perspective [27]–[29] views meta-learning as a tool to manage the ‘no free lunch’ theorem [30] and improve generalization by searching for the algorithm (inductive bias) that is best suited to a given problem, or problem family.

> 이러한 관점은 메타러닝을 '공짜 점심은 없다'는 정리(한계)에 대처하는 도구로 봅니다. 즉, 주어진 문제나 문제 그룹에 가장 적합한 알고리즘(귀납적 편향)을 탐색함으로써 일반화 성능을 향상시키는 도구라는 것입니다.

> However, this definition can include transfer, multi-task, feature-selection, and model-ensemble learning, which are not typically considered as meta-learning today.

> 하지만, 이러한 정의는 전이 학습, 다중과제 학습, 특징 선택, 모델 앙상블 학습까지 포함할 수 있는데, 이들은 오늘날 일반적으로 메타러닝으로 간주되지 않습니다.

+ "문제에 맞는 해결책을 찾는다"는 정의가 너무 광범위해서, 관련은 있지만 엄연히 다른 여러 기법들까지 전부 '메타러닝'의 범주에 포함시켜 버린다는 문제가 있다는 맥락입니다. 이 관점은 메타러닝의 철학을 이해하는 데는 도움이 되지만, 현대의 메타러닝을 다른 기법들과 구분 짓기에는 그 정의가 너무 모호하고 포괄적이라는 한계가 있습니다. 오늘날의 메타러닝은 단순히 알고리즘을 '선택'하거나 '재사용'하는 것을 넘어, '학습하는 과정 자체'를 최적화하는 더 구체적인 의미로 사용되기 때문입니다.

이 논문이 Review하려고 하는 대상 논문?

+ 본 논문에서는 현대의 신경망 기반 메타러닝에 집중하고 있습니다. 
  + 참고문헌 [27], [28]에 따라 **'알고리즘 학습'**으로 간주하지만, 특히 이 학습이 명시적으로 정의된 목적 함수(예: 교차 엔트로피 손실)의 종단간(end-to-end) 학습을 통해 달성되는 경우에 초점을 맞춥니다. 
  + 이에 더해, 우리는 단일 과제 메타러닝을 고려하고, 강건성(robustness) 및 컴퓨팅 효율성과 같은 더 폭넓고 다양한 (메타) 목적 함수에 대해서도 논의할 것입니다.

   
+ 본 논문에서는 메타러닝의 방법론과 응용 분야를 모두 다루려고 함. 
  + 먼저, 이 분야의 연구들을 이해하고 그 위상을 정립하는 데 사용할 수 있는 고수준의 문제 정형화(formalization)를 통해 메타러닝을 소개합니다
  + 그런 다음, **메타-표현(meta-representation), 메타-목적(meta-objective), 메타-최적화기(meta-optimizer)**라는 관점에서 새로운 분류 체계를 제공할 것입니다. 
  + 이 프레임워크는 새로운 메타러닝 방법을 개발하고 다양한 응용에 맞게 맞춤화하기 위한 **설계 공간(design-space)**을 제시합니다. 
  + 우리는 퓨샷 학습, 강화 학습, 구조 탐색 등 여러 인기 있고 새롭게 부상하는 응용 분야를 살펴보고, 전이 학습 및 다중과제 학습과 같은 관련 주제와 비교하여 메타러닝의 위상을 정립할 것입니다. 
  + 마지막으로, 아직 해결되지 않은 중요한 과제들과 미래 연구를 위한 유망한 영역들을 논의하며 마무리하겠습니다.


## 배경

메타러닝은 현대 신경망 문헌 내에서조차 
다양하고 일관성 없는 방식으로 사용되어 왔기 때문에 "한 단어, 한 문장"으로 명확하게 정의하기가 어려운 상태임. 

+ 이 섹션에서는 우리의 정의와 주요 용어를 소개하고, 관련 분야와 비교하여 메타러닝의 position을 정립하고 있음.


### 메타러닝?

**'학습하는 방법을 학습하는 것(learning to learn)'**

+ 이는 **여러 학습 에피소드에 걸쳐 학습 알고리즘을 개선하는 과정을 의미**함. 

+ 기존의 머신러닝이 여러 데이터 인스턴스에 걸쳐 모델 예측을 개선하는 것과 대조적. 


+ **기반 학습(base learning)** 중에는 **내부(inner)** (또는 하위/기반) 학습 알고리즘이 이미지 분류와 같은 과제를 해결하며, 이는 데이터셋과 목적 함수에 의해 정의됨. 흔히 inner loop라는 표현으로도 많이 쓰임    
+ **메타러닝** 중에는 **외부(outer)**(또는 상위/메타) 알고리즘이 내부 학습 알고리즘을 업데이트하여, 그것이 학습하는 모델이 외부 목적 함수를 개선하도록 만듦. 
  + 예를 들어, 이 목적 함수는 내부 알고리즘의 일반화 성능이나 학습 속도가 될 수 있습니다. (기반 알고리즘, 학습된 모델, 성능) 튜플로 구성된 기반 과제의 학습 에피소드들은 외부 알고리즘이 기반 학습 알고리즘을 학습하는 데 필요한 인스턴스를 제공하는 것으로 볼 수 있습니다.

```
During base learning, an inner (or lower/base) learning algorithm solves a task such as image classification [13], defined by a dataset and objective.

During meta-learning, an outer (or upper/meta) algorithm updates the inner learning algorithm such that the model it learns improves an outer objective.
```
> Base learning은 메타러닝의 **내부 루프(inner loop)** 에서 일어나는 개별적인 단일 과제(single task)에 대한 학습 과정을 의미함.
> 1.  **내부 수준 (Inner Level) = Base Learning (기반 학습)**
>
>    *   **목표**: 주어진 **하나의 특정 과제**를 해결하는 것 (예: '고양이'와 '개' 이미지를 분류하는 문제).
>    *   **구성 요소**: 이 과제를 해결하기 위한 데이터셋(고양이/개 이미지)과 목적 함수(분류 정확도를 높이는 손실 함수)가 주어집니다.
>    *   **알고리즘**: **`inner learning algorithm`** (기반 학습 알고리즘)이 이 데이터셋을 사용하여 모델 파라미터($\theta$)를 최적화합니다. 우리가 흔히 아는 경사 하강법(Gradient Descent) 같은 최적화 과정이 여기서 일어납니다.

> 2.  **외부 수준 (Outer Level) = Meta-Learning (메타러닝)**
>    *   **목표**: Base learning 자체를 **더 잘하게 만드는 것**입니다. 즉, 새로운 과제가 주어졌을 때 더 빠르거나, 더 적은 데이터로, 더 정확하게 학습할 수 있도록 **학습 알고리즘 자체($\omega$)를 개선**하는 것입니다.
>    *   **알고리즘**: **`outer algorithm`** (메타 학습 알고리즘)이 여러 번의 base learning 에피소드를 관찰하고, 그 결과를 바탕으로 `inner learning algorithm`을 업데이트합니다.


위에 정의된 대로라면, 교차 검증을 통한 하이퍼파라미터의 무작위 탐색과 같은 많은 기존 알고리즘이 메타러닝의 정의에 포함될 수 있습니다. 

현대 신경망 메타러닝의 두드러진 특징은 **명시적으로 정의된 메타 수준의 목적 함수**와, 이 목적 함수에 대한 내부 알고리즘의 **종단간(end-to-end) 최적화** 입니다. 
종종 메타러닝은 과제군(task family)에서 샘플링된 학습 에피소드에 대해 수행되어, 이 과제군에서 샘플링된 새로운 과제에 대해 좋은 성능을 보이는 기반 학습 알고리즘을 만듭니다. 
하지만, 극한의 경우 모든 훈련 에피소드가 단일 과제에서 샘플링될 수도 있습니다. 

---

### 2.1 메타러닝의 정형화

#### **기존 머신러닝** 

기존의 지도(supervised) 머신러닝에서는 (입력 이미지, 출력 레이블) 쌍과 같은 훈련 데이터셋 $D = \{(x_1, y_1), \dots, (x_N, y_N)\}$이 주어집니다. 
우리는 $\theta$로 매개변수화된 예측 모델 $\hat{y} = f_{\theta}(x)$를 다음 식을 풀어 훈련시킬 수 있습니다:


$$
\theta^* = \arg\min_{\theta} \mathcal{L}(D; \theta, \omega) \quad (1)
$$


* $\mathcal{L}$은 실제 레이블과 $f_{\theta}(\cdot)$가 예측한 값 사이의 오차를 측정하는 손실 함수입니다. 
* $\omega$에 대한 조건은 이 해법이 $\theta$에 대한 최적화기(optimizer)나 함수 $f$의 클래스 선택과 같은 '학습 방법'에 대한 가정에 의존함을 나타냅니다. 일반화 성능은 알려진 레이블을 가진 여러 테스트 포인트를 평가하여 측정됩니다.

기존의 가정은 이 최적화가 모든 문제 $D$에 대해 처음부터 수행되고, $\omega$는 사전에 지정된다는 것입니다. 그러나 $\omega$의 **사양**은 정확도나 데이터 효율성과 같은 성능 지표에 큰 영향을 미칠 수 있습니다. 
메타러닝은 이러한 지표를 개선하기 위해 학습 알고리즘 자체를 사전에 지정하고 고정하는 대신 학습함으로써 개선하고자 합니다. 이는 종종 위의 첫 번째 가정을 재검토하고, 처음부터 학습하는 대신 과제 분포로부터 학습함으로써 달성됩니다.

* 사양? 학습 알고리즘의 구체적인 내용과 구성이라고 할 수 있음

> 최적화기(Optimizer)의 종류: SGD, Adam, RMSprop 등 어떤 것을 쓸 것인가?    
> 학습률(Learning Rate): 학습률을 얼마로 설정할 것인가?   
> 모델 구조(Model Architecture): 어떤 종류의 신경망(CNN, RNN 등)을 사용할 것인가?   
> 정규화(Regularization) 방법: L1, L2, Dropout 등 어떤 정규화 기법을 적용할 것인가?   

#### 메타러닝: 과제 분포(Task-Distribution) 관점

메타러닝을 바라보는 가장 흔한 관점은, **여러 다양한 과제에 두루두루 잘 통하는 범용 학습 방법을 배우는 것**입니다. 이상적으로는, 새로운 과제를 배울 때마다 이전에 배운 경험 덕분에 더 잘 배우게 되는 것을 목표로 합니다.

어떤 학습 방법(ω)이 얼마나 좋은지를 평가하려면, 하나의 과제만 보는 것이 아니라 **과제들이 모여있는 전체 분포($p(\mathcal{T})$)**를 봐야 합니다. 여기서 '과제($\mathcal{T}$)'란 특정 '데이터셋($D$)'과 그 데이터셋으로 달성하려는 '목표(손실 함수 $L$)'의 조합이라고 생각할 수 있습니다. (예: '고양이/개 이미지 분류'라는 과제 = '고양이/개 이미지 데이터셋' + '분류 정확도를 높이는 손실 함수')

따라서 **"학습하는 법을 배운다"**는 것은, 결국 다음과 같은 목표를 달성하는 과정이 됩니다.

$$
\min_{\omega} \mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})} \mathcal{L}(D; \omega) \quad (2)
$$

> **[수식 (2) 해설]**   
> *   **ω (오메가)**: 우리가 찾고 싶은 '최고의 학습 전략' 또는 '학습 노하우'입니다.   
> *   **$\mathcal{T} \sim p(\mathcal{T})$**: 세상에 존재할 수 있는 수많은 과제들 중에서 무작위로 하나를 뽑는다는 뜻입니다.    
> *   **$\mathbb{E}$**: 평균적으로(기댓값)라는 의미입니다.   
> *   **$\mathcal{L}(D; \omega)$**: 학습 전략(ω)을 사용해서 어떤 과제의 데이터(D)를 학습했을 때의 최종 성능(손실 값)입니다.    
>
> **전체 의미**: "어떤 과제가 주어지든 **평균적으로 가장 좋은 성능을 내는 만능 학습 전략(ω)을 찾아라!**" 라는 뜻입니다. 이 학습 전략(ω)을 **메타 지식(meta-knowledge)** 이라고 부릅니다.   
> * 또는 `across-task knowledge`라고 하기도 함.

실제로는 세상의 모든 과제에 접근할 수 없으므로, 우리는 **샘플 과제들**을 가지고 학습을 진행합니다. 이 과정을 두 단계로 나눌 수 있습니다.

---


#### 1단계: 메타-훈련 (Meta-Training) - "학습 노하우 배우기"

먼저, '훈련용'으로 사용할 M개의 샘플 과제 묶음 $\mathcal{D}_{\text{source}}$를 준비합니다. 각 과제는 두 종류의 데이터로 나뉩니다.

*   **훈련용 데이터 ($D_{\text{train}}$)**: 이 과제를 '학습'하는 데 사용됩니다. 메타러닝에서는 이를 **서포트셋(support set)** 이라고 부릅니다.
*   **검증용 데이터 ($D_{\text{val}}$)**: 위에서 학습이 얼마나 잘 되었는지 '평가'하는 데 사용됩니다. 메타러닝에서는 이를 **쿼리셋(query set)** 이라고 부릅니다.

이 훈련용 과제 묶음을 가지고 "학습 노하우를 배우는" 메타-훈련 단계는 다음과 같이 표현됩니다.

$$
\omega^* = \arg\max_{\omega} \log p(\omega|\mathcal{D}_{\text{source}}) \quad (3)
$$

> **[수식 (3) 해설]**   
> 이 단계의 목표는 "주어진 훈련용 과제들($\mathcal{D}_{\text{source}}$)을 가장 잘 해결할 수 있게 해주는 **최고의 학습 전략($\omega^*$)**을 찾는 것"입니다. 비유하자면, 여러 학교의 기출문제들을 풀어보면서 어떤 시험에도 통할 '만능 공부법'을 터득하는 과정입니다.

---

#### 2단계: 메타-테스트 (Meta-Testing) - "배운 노하우로 실전 문제 풀기"

이제 학습이 끝났으니, 한 번도 본 적 없는 '실전용' 타겟 과제 묶음 $\mathcal{D}_{\text{target}}$으로 테스트를 합니다. 각 타겟 과제($i$)에 대해, 우리는 1단계에서 배운 **최고의 학습 전략($\omega^*$)**을 사용하여 모델을 훈련시킵니다.

$$
\theta^{*(i)} = \arg\max_{\theta} \log p(\theta|\omega^*, D_{\text{train}}^{\text{target}}(i)) \quad (4)
$$

> **[수식 (4) 해설]**
> 기존의 학습(수식 1)과 가장 큰 차이점은 **$\omega^*$가 추가되었다**는 점입니다.
>
> *   기존 학습: 백지상태에서 특정 과제($D_{\text{train}}$)를 열심히 풀어 모델 파라미터($\theta$)를 찾습니다.
> *   메타 학습 후: 우리가 미리 터득한 '만능 공부법'($\omega^*$)을 **바탕으로** 새로운 과제($D_{\text{train}}^{\text{target}}$)를 훨씬 효율적으로 풀어 모델 파라미터($\theta^*$)를 찾습니다.
>
> 이 $\omega^*$는 좋은 **출발점(초기 파라미터)**이 될 수도 있고, 아예 새로운 **모델 자체**나 **최적화 방법**이 될 수도 있습니다.

최종적으로 메타러닝이 얼마나 성공적이었는지는, 이렇게 훈련된 모델($\theta^{*(i)}$)이 실전 문제의 테스트 데이터($D_{\text{test}}^{\text{target}}(i)$)에서 얼마나 높은 점수를 받는지로 평가합니다.

---

#### 메타-과적합 (Meta-Overfitting) 문제

일반적인 머신러닝에 과소적합/과적합이 있듯이, 메타러닝에도 **메타-과소적합**과 **메타-과적합**이 있습니다. 특히 메타-과적합은 중요한 문제입니다.

**메타-과적합이란?**
훈련용 과제들(source tasks)에서 배운 학습 전략($\omega$)이 너무 그 훈련용 과제들에만 특화되어서, 막상 새로운 실전 과제들(target tasks)에는 전혀 통하지 않는 현상입니다.

> **[쉬운 예시]**
> 수학 공부를 할 때 '대수학' 기출문제만 잔뜩 풀어서 "모든 문제는 인수분해로 풀면 된다!"는 식의 편협한 공부법($\omega$)을 터득했다고 상상해보세요. 이 학생은 대수학 문제에서는 최고의 성적을 낼 것입니다. 하지만 갑자기 '기하학' 시험을 보게 되면, 인수분해 전략이 통하지 않아 완전히 망치게 됩니다. 이것이 바로 **메타-과적합**입니다. 너무 좁은 범위의 문제에만 통하는 '학습 전략'을 배운 탓입니다.